In [2]:
#BIBLIOTEKI I OPCJE
import pandas as pd
import re
import random
import math
import uuid
import datetime
import numpy as np

import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

#TABELE
# playerTable = pd.DataFrame(columns=['Name', 'Surname', 'City', 'Category']) #to provide player names
# playerTable = pd.read_csv('examplePlayerTable.csv')
#df = pd.read_csv('csv/examplePlayerTable.csv')
#trickTable = pd.read_csv('trickTableJapan.csv')
#df = df.loc[df.index <= 10]
df = pd.read_csv('csv/masterTable.csv')
#MODUŁY
%run tableFunction.ipynb
%run gameFunction.ipynb 

In [3]:
playersDict = {
    'Jack': {
        'skills': 3,
        'freq': 9
    },
    'Mike': {
        'skills': 5,
        'freq': 8
    },
    'Anna': {
        'skills': 3,
        'freq': 5
    },
    'Braina': {
        'skills': 5,
        'freq': 5
    },
    'Jessica': {
        'skills': 4,
        'freq': 7
    },
    'Sonia': {
        'skills': 6,
        'freq': 3
    },
}

df = pd.DataFrame(columns=[['name', 'gameUID', 'gamesWin', 'gamesTotal', 'masterUID']])


def devFreq(playersDict):
    for p in playersDict:
        playerFreq = playersDict[p].get('freq')
        playerFreq = playerFreq * random.randint(1,6)
        playersDict[p].update({'gameFreq': playerFreq})
    return playersDict

def devSkill(playersDict, randomFacto=True):
    for p in playersDict:
        playerSkill = playersDict[p].get('skills')
        playerSkill = playerSkill * random.randint(1,6)
        if randomFacto == True: playerSkill += random.randint(-5,5)
        playersDict[p].update({'gameSkills': playerSkill})
    return playersDict

def devGame(playersDict, pArray=[]):
    if len(pArray) == 0:
        pDict = devFreq(playersDict)
        pDict = devSkill(playersDict)
        gamePopulation = random.randint(2,len(pDict)) 
        
        pFreq = []
        for p in pDict: pFreq.append(pDict[p].get('gameFreq'))
        while len(pFreq) > gamePopulation: pFreq.pop(pFreq.index(min(pFreq)))
        
        game = {
            'name': [],
            'winner': [],
            'points': []
        } 
    
                  
    
        for p in pDict: 
            if pDict[p].get('gameFreq') in pFreq: game['name'].append(p)

    elif len(pArray) > 1:
        pDict = devSkill(playersDict)
        
        game = {
            'name': pArray,
            'winner': [],
            'points': []
        } 
            
    pSkill = []
    for p in game['name']: pSkill.append(pDict[p].get('gameSkills'))
    for p in game['name']: 
        if pDict[p].get('gameSkills') == max(pSkill) and game['winner'].count(True) == 0: game['winner'].append(True)
        else: game['winner'].append(False)

    for p in game['name']:
        pIndex = game['name'].index(p)
        if game['winner'][pIndex] == True: game['points'].append(random.randint(0,2))
        else: game['points'].append(3)
    
    return game

def devSession(maxRounds=6):
    gameRounds = random.randint(1,maxRounds)
    game = devGame(playersDict)
    
    game.update({'gameUID': 0})
    dfN = pd.DataFrame.from_dict(game)
    
    roundsLeft = gameRounds-1
    if gameRounds > 1:
        while roundsLeft != 0:
            nextGame = devGame(playersDict, pArray=game['name'])
            nextGame.update({'gameUID': roundsLeft})
            dfNext = pd.DataFrame.from_dict(nextGame)
            dfN = pd.concat([dfN, dfNext])
            roundsLeft -= 1
            
    masterUID = uuid.uuid4().time
    dfN['masterUID'] = masterUID
    return dfN

def devGenerateTable(sessionNumber=20):
    n = 1
    df = devSession()
    while n < sessionNumber:
        dfT = devSession()
        df = pd.concat([df, dfT])
        n += 1
    for n in df['masterUID']:
        dfN = df.loc[df['masterUID'] == n]
        dfCount = dfN['gameUID'].nunique()
        df.loc[df['masterUID'] == n, 'finGames'] = dfCount
    df['finGames'] = df['finGames'].astype(int)
    df.to_csv('csv/playerModel.csv', index=False)
devGenerateTable(sessionNumber=20)

In [166]:
dfTest = pd.read_csv('csv/playerModel.csv')

In [168]:
dfTest.head(60)

,name,winner,points,gameUID,masterUID,finGames
0,Jack,False,3,0,441621366102261820,5
1,Mike,False,3,0,441621366102261820,5
2,Anna,False,3,0,441621366102261820,5
3,Braina,False,3,0,441621366102261820,5
4,Jessica,False,3,0,441621366102261820,5
5,Sonia,True,2,0,441621366102261820,5
6,Jack,False,3,4,441621366102261820,5
7,Mike,False,3,4,441621366102261820,5
8,Anna,False,3,4,441621366102261820,5
9,Braina,False,3,4,441621366102261820,5


In [128]:
#filtr czasowy
def filterByDate(df, startDate=0, startTime='00:00', endDate=0, endTime='23:59'):
    if startDate == 0: startDate = df['timestamp'].values[0]
    if endDate == 0: endDate = df['timestamp'].values[-1]
    df['timestamp'] = pd.to_datetime(df['timestamp'], yearfirst=True)
    # startTime = '00:00'
    # startDate = '2022-02-01'
    # endTime = '23:59'
    # endDate = '2022-02-01'
    df = df.loc[(df['timestamp'] >= f'{startDate} {startTime}') & (df['timestamp'] <= f'{endDate} {endTime}')]
    return df

# osoba, która wygrała najwięcej gier
def overallWinner(df):
    df = df.loc[df['end'] == False]
    dfN = df[['name', 'end']].groupby('name').count().sort_values('end', ascending=False)
    dfDict = dfN.to_dict()
    return dfDict

#osoba, która wygrała najwięcej w danym czasie
def timeWinner(df, startDate=0, startTime='00:00', endDate=0, endTime='23:59'):
    print(startDate)
    dfFilter = filterByDate(df, startDate, startTime, endDate, endTime)
    dfDict = overallWinner(dfFilter)
    return dfDict

#filtr po imieniu
def filterByName(df, playerName):
    dfM = df.loc[df['name'] == playerName]
    dfMap = set(dfM['masterUID'])
    df = df.loc[df['masterUID'].isin(dfMap)]
    dfDict = df.to_dict('records')
    return dfDict


In [ ]:
df.loc[df['masterUID'] == '716597bf-bab6-453f-a136-d59a25b263e9']

In [ ]:
df.value_counts('name')